In [ ]:
# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import librosa
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
# Emotion mapping
emotion_map = {
    '01': 'neutral',
    '02': 'calm',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry',
    '06': 'fearful',
    '07': 'disgust',
    '08': 'surprised'
}

# Optional: Statement mapping
statement_map = {
    '01': "Kids are talking by the door",
    '02': "Dogs are sitting by the door"
}


In [ ]:
def extract_features_from_directory(data_dir, modality_label):
    data = []

    for actor_folder in sorted(os.listdir(data_dir)):
        actor_path = os.path.join(data_dir, actor_folder)
        if not os.path.isdir(actor_path):
            continue

        for file in tqdm(os.listdir(actor_path), desc=f"Processing {actor_folder}"):
            if not file.endswith(".wav"):
                continue

            file_path = os.path.join(actor_path, file)
            parts = file.strip(".wav").split("-")

            if len(parts) != 7:
                continue  # skip malformed filenames

            modality = parts[0]
            vocal_channel = parts[1]
            emotion_code = parts[2]
            intensity = parts[3]
            statement = parts[4]
            repetition = parts[5]
            actor_id = int(parts[6].split()[0])  # removes anything after space like '(1)'


            emotion = emotion_map.get(emotion_code)
            gender = "male" if actor_id % 2 != 0 else "female"

            # Load audio
            try:
                y, sr = librosa.load(file_path, sr=16000)
                y = librosa.util.normalize(y)
                y, _ = librosa.effects.trim(y, top_db=30)

                # Extract MFCC
                mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
                mfcc = mfcc.T  # Keep time steps!

                data.append({
                    "features": mfcc,
                    "emotion": emotion,
                    "modality": modality_label,
                    "intensity": intensity,
                    "statement": statement_map.get(statement),
                    "repetition": repetition,
                    "actor": actor_id,
                    "gender": gender,
                    "filename": file_path
                })

            except Exception as e:
                print(f"Error processing {file_path}: {e}")
                continue

    return pd.DataFrame(data)


In [ ]:
# Define paths in your Drive
speech_dir = "/content/drive/MyDrive/RAVDESS/Audio_Speech_Actors_01-24"
song_dir = "/content/drive/MyDrive/RAVDESS/Audio_Song_Actors_01-24"

# Extract and label modality
df_speech = extract_features_from_directory(speech_dir, modality_label="speech")
df_song = extract_features_from_directory(song_dir, modality_label="song")

# Combine
df_combined = pd.concat([df_speech, df_song], ignore_index=True)

# Show preview
df_combined.head()


Processing Actor_24: 100%|██████████| 44/44 [00:00<00:00, 44.87it/s]


,features,emotion,modality,intensity,statement,repetition,actor,gender,filename
0,"[[-418.6446, 12.663326, 4.063743, 42.066853, -...",calm,speech,01,Kids are talking by the door,01,1,male,/content/drive/MyDrive/RAVDESS/Audio_Speech_Ac...
1,"[[-361.14465, 33.370667, 49.87454, 44.46737, -...",calm,speech,02,Kids are talking by the door,01,1,male,/content/drive/MyDrive/RAVDESS/Audio_Speech_Ac...
2,"[[-397.96228, 30.884481, 8.242433, 39.433132, ...",neutral,speech,01,Dogs are sitting by the door,02,1,male,/content/drive/MyDrive/RAVDESS/Audio_Speech_Ac...
3,"[[-278.17993, 8.254922, -5.0730753, 38.98133, ...",neutral,speech,01,Kids are talking by the door,02,1,male,/content/drive/MyDrive/RAVDESS/Audio_Speech_Ac...
4,"[[-353.22528, 20.584011, -21.754581, 43.88891,...",happy,speech,01,Kids are talking by the door,01,1,male,/content/drive/MyDrive/RAVDESS/Audio_Speech_Ac...


In [ ]:
# Save as pickle (preserves numpy arrays inside the DataFrame)
df_combined.to_pickle("/content/drive/MyDrive/ravdess_mfcc_df_fullseq.pkl")

# Or save as CSV (converts MFCCs to string — less ideal)
# df_combined.to_csv("/content/drive/MyDrive/ravdess_mfcc_df.csv", index=False)


In [ ]:
df_combined.head()

,features,emotion,modality,intensity,statement,repetition,actor,gender,filename
0,"[[-418.6446, 12.663326, 4.063743, 42.066853, -...",calm,speech,01,Kids are talking by the door,01,1,male,/content/drive/MyDrive/RAVDESS/Audio_Speech_Ac...
1,"[[-361.14465, 33.370667, 49.87454, 44.46737, -...",calm,speech,02,Kids are talking by the door,01,1,male,/content/drive/MyDrive/RAVDESS/Audio_Speech_Ac...
2,"[[-397.96228, 30.884481, 8.242433, 39.433132, ...",neutral,speech,01,Dogs are sitting by the door,02,1,male,/content/drive/MyDrive/RAVDESS/Audio_Speech_Ac...
3,"[[-278.17993, 8.254922, -5.0730753, 38.98133, ...",neutral,speech,01,Kids are talking by the door,02,1,male,/content/drive/MyDrive/RAVDESS/Audio_Speech_Ac...
4,"[[-353.22528, 20.584011, -21.754581, 43.88891,...",happy,speech,01,Kids are talking by the door,01,1,male,/content/drive/MyDrive/RAVDESS/Audio_Speech_Ac...
